<a href="https://colab.research.google.com/github/Roterdamjr/ControleFinanceiro/blob/main/Controle_Financeiro_Python_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biblioteca

> Adicionar aspas



In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from io import BytesIO
import yfinance as yf
import xlsxwriter

def load_data(excel_filepath):

    df_patrimonio = fn_busca_aba_excel(excel_filepath,'Patrimonio')
    df_alocacao_global = fn_busca_aba_excel(excel_filepath,'Alocacao global')
    df_cadastro_original_acoes = fn_busca_aba_excel(excel_filepath,'Cadastro Original Acoes')
    df_historico = fn_busca_aba_excel(excel_filepath,'Historico')
    df_cadastro_fi = fn_busca_aba_excel(excel_filepath,'Cadastro FI')
    df_cadastro_recebiveis = fn_busca_aba_excel(excel_filepath,'Cadastro Recebiveis')
    df_renda_fixa = fn_busca_aba_excel(excel_filepath,'Renda Fixa')
    df_exterior = fn_busca_aba_excel(excel_filepath,'Exterior')

    df_proporcoes_dh_dp =pd.read_excel(excel_filepath, sheet_name='Proporcoes DH_DP',  engine='openpyxl' )
    proporcao_dh = df_proporcoes_dh_dp['DH'].iloc[0] if not df_proporcoes_dh_dp['DH'].empty else 0.0
    proporcao_dp = df_proporcoes_dh_dp['DP'].iloc[0] if not df_proporcoes_dh_dp['DP'].empty else 0.0

    return (df_patrimonio, df_alocacao_global, df_cadastro_original_acoes, df_historico, df_cadastro_fi,
            df_cadastro_recebiveis, df_renda_fixa,df_exterior,
            proporcao_dh, proporcao_dp)

def fn_busca_aba_excel(excel_filepath,nome_aba ):
    df_aba = pd.read_excel(
        excel_filepath,
        sheet_name=nome_aba,
        engine='openpyxl'
    )

    df_aba = df_aba.dropna(axis=1,how='all')
    df_aba.set_index('Ativo', inplace=True)
    return df_aba

def fn_busca_pm_quantidade(df):

  #recebe dados de uma ativo e deveolve preco médio, quantidade eresultado da vendas
  #df_vendas = pd.DataFrame(columns=['Data', 'Quantidade', 'Preço Compra', 'Preço Venda'])
  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return qtde_total,preco_medio

def fn_calc_metricas(nome_ativo,
                     df_historico,
                     df_alocacao,
                     df_cotacoes ):

  ##############################
  # calcula quantidade, preco_medio, cotacao ,
  # patrimonio, resultado, rentabilidade, alocacao
  ##############################
  cotacao = df_cotacoes[nome_ativo]

  if nome_ativo in df_historico.index:
    df_ativo = df_historico.loc[nome_ativo]
    quantidade, preco_medio = fn_busca_pm_quantidade(df_ativo)

    alocacao = df_alocacao.loc[nome_ativo]['Alocacao']
    patrimonio = quantidade * cotacao
    resultado = patrimonio - quantidade * preco_medio
    rentabilidade = resultado / (quantidade * preco_medio)
  else:
      quantidade = 0
      preco_medio= 0
      alocacao = 0
      patrimonio = 0
      resultado = 0
      rentabilidade = 0

  return [int(quantidade),
          float(round(preco_medio, 2)),
          float(round(cotacao, 2)) ,
          float(round(patrimonio, 2)),
          float(round(resultado, 2)),
          float(round(rentabilidade, 2)),
          float(round(alocacao, 3))]

def fn_gera_carteira( p_df_historico, p_df_cadastro, p_df_cotacoes):
  df_carteira = pd.DataFrame([fn_calc_metricas(ativo, p_df_historico, p_df_cadastro, p_df_cotacoes)
                              for ativo in p_df_cadastro.index],
                          columns=['Quantidade', 'Preço Médio', 'Cotação', 'Patrimônio', 'Resultado', 'Rentabilidade', 'Alocacao'],
                          index=p_df_cadastro.index)

  # Totais
  patrimonio_total, resultado_total = df_carteira['Patrimônio'].sum(), df_carteira['Resultado'].sum()

  return df_carteira

def fn_busca_cotacoes(p_ativos):
  df_cotacoes = pd.DataFrame()
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
  df_cotacoes.index = df_cotacoes.index.str.replace('.SA', '', regex=False)
  return df_cotacoes

def fn_object_para_numerico(df,nome_coluna):
    df[nome_coluna] = df[nome_coluna].str.replace('.', '', regex=False)
    df[nome_coluna] = df[nome_coluna].str.replace(',', '.', regex=False).astype(float)


def fn_percet_para_numerico(df, nome_coluna):

  coluna_limpa = (
          df[nome_coluna]
          .astype(str)
          .str.replace('%', '', regex=False)
          .str.replace(',', '.', regex=False)
      )

  df['Alocacao'] = pd.to_numeric(coluna_limpa, errors='coerce') / 100



def fn_exporta(dataframes_dict, nome_arquivo):

    writer = pd.ExcelWriter(nome_arquivo, engine='xlsxwriter')
    workbook = writer.book

    # Definição de Cores e Formatos Base
    COR_CINZA = '#A9A9A9'
    COR_BEGE_CLARO = '#F5F5DC'
    COR_BEGE_PADRAO = '#F0EAD6'
    COR_VERDE = '#90EE90'
    BORDA = 1

    FORMATO_MOEDA = 'R$ #,##0.00'
    FORMATO_PERCENT = '0.00%'

    base_body_format = {'fg_color': COR_BEGE_PADRAO, 'border': BORDA}

    custom_formats_map = {
        'Ativo': (COR_BEGE_CLARO, None),
        'Patrimonio': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Preço Médio': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Cotação': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Resultado': (COR_BEGE_PADRAO, FORMATO_MOEDA),
        'Aportar': (COR_VERDE, FORMATO_MOEDA),
        'Rentabilidade': (COR_BEGE_PADRAO, FORMATO_PERCENT),
        'Alocacao': (COR_BEGE_PADRAO, FORMATO_PERCENT),
    }

    formatos_excel = {}
    for col_name, (cor_fundo, num_format) in custom_formats_map.items():
        fmt_dict = {'fg_color': cor_fundo, 'border': BORDA}
        if num_format:
            fmt_dict['num_format'] = num_format
        formatos_excel[col_name] = workbook.add_format(fmt_dict)

    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': COR_CINZA,
        'border': BORDA
    })

    default_format = workbook.add_format(base_body_format)

    # Itera sobre cada DataFrame e Cria uma Nova Aba
    for sheet_name, df in dataframes_dict.items():

        df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=True)

        worksheet = writer.sheets[sheet_name]
        colunas_excel = [df.index.name or 'Ativo'] + list(df.columns.values)

        # Escreva o Cabeçalho e Defina a Largura das Colunas
        for col_num, value in enumerate(colunas_excel):
            worksheet.write(0, col_num, value, header_format)

            if value in ['Ativo', df.index.name]:
                width = 15
            elif value in ['Alocacao', 'Tipo']:
                width = 10
            elif value in ['Quantidade', 'Preço Médio', 'Cotação', 'Resultado']:
                width = 15
            else:
                width = 18

            worksheet.set_column(col_num, col_num, width)

        # Aplique a formatação no corpo da tabela
        num_linhas = len(df)
        for row in range(1, num_linhas + 1):

            for col_num, col_name in enumerate(colunas_excel):

                if col_name == (df.index.name or 'Ativo'):
                    valor = df.index[row - 1]
                else:
                    valor = df.iloc[row - 1][col_name]

                formato = formatos_excel.get(col_name, default_format)

                worksheet.write(row, col_num, valor, formato)

    # Salve o arquivo
    writer.close()



In [ ]:
def fn_vendas(df_hist):

  # Recebe dados de uma ativo e deveolve preco médio, quantidade das  vendas

  a_vendas = []

  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df_hist.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']
      data= linha['Data']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
          a_vendas.append([indice,data, qtde,valor,preco_medio])
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return a_vendas

# Teste

In [ ]:
def fn_vendas(df_hist):

  # Recebe dados de uma ativo e deveolve preco médio, quantidade das  vendas

  a_vendas = []

  preco_medio, qtde_total = 0.0, 0

  for indice, linha in df_hist.iterrows():
      valor = linha['Valor']
      qtde = linha['Quant']
      operacao = linha['Operação']
      data= linha['Data']

      if operacao == 'C':
          preco_medio = (preco_medio * qtde_total + valor * qtde) / (qtde_total + qtde)
          qtde_total += qtde
      elif operacao == 'V':
          qtde_total -= qtde
          a_vendas.append([indice,data, qtde,valor,preco_medio])
      elif operacao == 'A':
          preco_medio = (preco_medio * qtde_total) / (qtde_total - qtde)
          qtde_total -= qtde
      elif operacao == 'D':
          preco_medio = (preco_medio * qtde_total) / (qtde_total + qtde)
          qtde_total += qtde

  return a_vendas

####################################
#     Download da Planilha
####################################
df_patrimonio, df_alocacao_global, df_cadastro_original_acoes, df_historico, df_cadastro_fi, \
            df_cadastro_recebiveis, df_renda_fixa,df_exterior,\
            proporcao_dh, proporcao_dp = load_data('/content/Controle Financeiro.xlsx' )

#####################
a_vendas_total = []

for nome_ativo in df_cadastro_original_acoes.index:

    if nome_ativo in df_historico.index:
        df_hist_ativo = df_historico.loc[nome_ativo]
        a_vendas = fn_vendas(df_hist_ativo)
        a_vendas_total = a_vendas_total + a_vendas

df_vendas = pd.DataFrame(a_vendas_total, columns=['Ativo', 'Data', 'Qtde Vendida', 'Preço Venda', 'Preço Compra'])
df_vendas['Resultado'] = df_vendas['Qtde Vendida'] * (df_vendas['Preço Venda'] - df_vendas['Preço Compra'])
df_vendas

,Ativo,Data,Qtde Vendida,Preço Venda,Preço Compra,Resultado
0,ALOS3,2023-05-19,200,20.93,18.380000,510.000000
1,ALOS3,2024-08-26,100,23.93,22.799767,113.023256
2,ALOS3,2024-08-30,200,23.08,22.799767,56.046512
3,BBAS3,2023-05-02,30,42.15,32.090000,301.800000
4,BBAS3,2023-06-12,40,49.30,32.090000,688.400000
...,...,...,...,...,...,...
59,CXSE3,2024-12-16,100,15.69,13.222800,246.720000
60,CXSE3,2025-10-07,100,15.20,14.095800,110.420000
61,TAEE11,2025-09-25,100,36.41,33.741818,266.818182
62,TAEE11,2025-11-07,100,40.29,33.741818,654.818182


# Planilha e cotacoes

In [ ]:
####################################
#     Download da Planilha
####################################
df_patrimonio, df_alocacao_global, df_cadastro_original_acoes, df_historico, df_cadastro_fi, \
            df_cadastro_recebiveis, df_renda_fixa,df_exterior,\
            proporcao_dh, proporcao_dp = load_data('/content/Controle Financeiro.xlsx' )

In [ ]:
####################################
#     Download de Cotações
####################################
lista_de_ativos = df_cadastro_original_acoes.index.tolist()
lista_de_ativos.extend(df_cadastro_fi.index.tolist())
lista_de_ativos.extend(df_cadastro_recebiveis.index.tolist())
df_cotacoes = fn_busca_cotacoes(lista_de_ativos)

/tmp/ipython-input-2303118951.py:110: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_cotacoes = yf.download([at +".SA" for at in p_ativos])['Close'].iloc[-1]
[*********************100%***********************]  32 of 32 completed


#  Cadastro ponderado de acoes, FI e Recebiveis

In [ ]:
################################################################
# Geracao de df_cadastro_acoes_ponderado
# que se baeia nos pesos do Dica Acoes e Dica Prev
################################################################
df_acoes_dh = df_cadastro_original_acoes.loc[df_cadastro_original_acoes['Tipo'] == 'DH'].copy()
soma_total = df_acoes_dh['Alocacao'].sum()
df_acoes_dh['Calculado'] = df_acoes_dh['Alocacao'] / soma_total * proporcao_dh

df_acoes_dp = df_cadastro_original_acoes.loc[df_cadastro_original_acoes['Tipo'] == 'DP'].copy()
soma_total = df_acoes_dp['Alocacao'].sum()
df_acoes_dp['Calculado'] = df_acoes_dp['Alocacao'] / soma_total * proporcao_dp

df_cadastro_acoes_ponderado = pd.concat([df_acoes_dh, df_acoes_dp]).sort_index()
df_cadastro_acoes_ponderado.drop('Alocacao', axis=1, inplace=True)
df_cadastro_acoes_ponderado.rename(columns={'Calculado': 'Alocacao'}, inplace=True)
df_cadastro_acoes_ponderado['Alocacao'] = df_cadastro_acoes_ponderado['Alocacao'].round(3)

####################################
#  Geração das carteiras de Ações,
#  Fundos imobiliarios e Recebiveis
####################################
df_carteira_acoes = fn_gera_carteira  (df_historico, df_cadastro_acoes_ponderado, df_cotacoes)
df_carteira_acoes['Preco Justo'] = df_cadastro_acoes_ponderado['Preco Justo']

df_carteira_fi = fn_gera_carteira  (df_historico, df_cadastro_fi, df_cotacoes)
df_carteira_recebiveis = fn_gera_carteira  (df_historico, df_cadastro_recebiveis, df_cotacoes)


# Carteira Estruturada Global

In [ ]:
df_carteira_global  = pd.DataFrame(columns=['Ativo', 'Patrimonio', 'Alocacao', 'Aportar']).set_index('Ativo')

linhas_a_atualizar = ['Acoes', 'Fundos Imobiliarios','Recebiveis','CDB Dan','CDB Marcia',
                      'Pre-Fixado','Selic','IPCA', 'VGBL Dan','VGBL Marcia', 'Exterior', 'Opcoes']

df_carteira_global = df_carteira_global.reindex(df_carteira_global.index.union(linhas_a_atualizar))
df_carteira_global.loc[linhas_a_atualizar, 'Patrimonio'] = df_patrimonio['Patrimonio'].loc[linhas_a_atualizar]

#### Alocação ###
df_carteira_global.loc['Acoes', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Acoes']
df_carteira_global.loc['Fundos Imobiliarios', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['FII']
df_carteira_global.loc['Recebiveis', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Recebiveis']

fator_base = df_alocacao_global.loc['Renda Fixa', 'Alocacao']
indices_rf = df_renda_fixa.index
df_carteira_global.loc[indices_rf, 'Alocacao'] = (
    round(fator_base * df_renda_fixa['Alocacao'],3)
)

df_carteira_global.loc['Exterior', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Exterior']
df_carteira_global.loc['Opcoes', 'Alocacao'] = df_alocacao_global ['Alocacao'].loc['Opcoes']

df_carteira_global['Aportar'] = df_carteira_global['Alocacao'] * df_carteira_global['Patrimonio'].sum() - df_carteira_global['Patrimonio']

###### calcula e atualiza coluna Aportar        #####
###### em na carteira de Acoes, FI e Recebiveis #####
tot = df_carteira_acoes['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Acoes']
df_carteira_acoes['Aportar'] =  df_carteira_acoes['Alocacao'] * (tot + aporte) - df_carteira_acoes['Patrimônio']

tot = df_carteira_fi['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Fundos Imobiliarios']
df_carteira_fi['Aportar'] =  df_carteira_fi['Alocacao'] * (tot + aporte) - df_carteira_fi['Patrimônio']

tot = df_carteira_recebiveis['Patrimônio'].sum()
aporte = df_carteira_global['Aportar'].loc['Recebiveis']
df_carteira_recebiveis['Aportar'] =  df_carteira_recebiveis['Alocacao'] * (tot + aporte) - df_carteira_recebiveis['Patrimônio']


# Imposto de Renda

In [ ]:
a_vendas_total = []

for nome_ativo in df_cadastro_original_acoes.index:

    if nome_ativo in df_historico.index:
        df_hist_ativo = df_historico.loc[nome_ativo]
        a_vendas = fn_vendas(df_hist_ativo)
        a_vendas_total = a_vendas_total + a_vendas

df_vendas = pd.DataFrame(a_vendas_total, columns=['Ativo', 'Data', 'Qtde Vendida', 'Preço Venda', 'Preço Compra'])
df_vendas['Resultado'] = round(
      df_vendas['Qtde Vendida'] * (df_vendas['Preço Venda'] - df_vendas['Preço Compra']),
      2)
df_vendas['Valor Venda'] = df_vendas['Qtde Vendida'] * df_vendas['Preço Venda']

df_vendas

,Ativo,Data,Qtde Vendida,Preço Venda,Preço Compra,Resultado,Valor Venda
0,ALOS3,2023-05-19,200,20.93,18.380000,510.00,4186.0
1,ALOS3,2024-08-26,100,23.93,22.799767,113.02,2393.0
2,ALOS3,2024-08-30,200,23.08,22.799767,56.05,4616.0
3,BBAS3,2023-05-02,30,42.15,32.090000,301.80,1264.5
4,BBAS3,2023-06-12,40,49.30,32.090000,688.40,1972.0
...,...,...,...,...,...,...,...
59,CXSE3,2024-12-16,100,15.69,13.222800,246.72,1569.0
60,CXSE3,2025-10-07,100,15.20,14.095800,110.42,1520.0
61,TAEE11,2025-09-25,100,36.41,33.741818,266.82,3641.0
62,TAEE11,2025-11-07,100,40.29,33.741818,654.82,4029.0


In [ ]:
df_irrf = df_vendas[['Data','Resultado','Valor Venda']].copy()
df_irrf['mes_ano'] = df_vendas['Data'].dt.to_period('M')
df_irrf.drop(['Data'], axis=1, inplace=True)
df_irrf = df_irrf.groupby('mes_ano').sum()
df_irrf

,Resultado,Valor Venda
mes_ano,,
2023-05,1675.20,15354.5
2023-06,1181.40,6352.0
2023-11,224.19,2090.0
2024-01,537.19,2403.0
2024-02,602.82,4965.0
2024-03,2195.79,10940.4
2024-06,6.49,9531.5
2024-08,169.07,7009.0
2024-11,106.58,1105.0


# Exportação

In [ ]:
# Crie um dicionário com todos os DataFrames e os nomes desejados para as abas
# Substitua df_carteira_acoes, df_carteira_fi, etc. pelos seus DataFrames reais.
dfs_to_export = {
    'Global': df_carteira_global,
    'Ações': df_carteira_acoes,
    'FII': df_carteira_fi,
    'Recebíveis': df_carteira_recebiveis
}

# Exporta todos para um único arquivo
fn_exporta(dfs_to_export, 'Carteira Global de Investimentos.xlsx')